In [1]:
#数据
my_data=[['slashdot','USA','yes',18,'None'],

['google','France','yes',23,'Premium'],

['digg','USA','yes',24,'Basic'],

['kiwitobes','France','yes',23,'Basic'],

['google','UK','no',21,'Premium'],

['(direct)','New Zealand','no',12,'None'],

['(direct)','UK','no',21,'Basic'],

['google','USA','no',24,'Premium'],

['slashdot','France','yes',19,'None'],

['digg','USA','no',18,'None'],

['google','UK','no',18,'None'],

['kiwitobes','UK','no',19,'None'],

['digg','New Zealand','yes',12,'Basic'],

['slashdot','UK','no',21,'None'],

['google','UK','yes',18,'Basic'],

['kiwitobes','France','yes',19,'Basic']]

In [2]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.tree import DecisionTreeClassifier as DTC

D:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
data = pd.DataFrame(my_data)

In [4]:
data_targets = data[data.columns[4]]

In [5]:
data_targets

0        None
1     Premium
2       Basic
3       Basic
4     Premium
5        None
6       Basic
7     Premium
8        None
9        None
10       None
11       None
12      Basic
13       None
14      Basic
15      Basic
Name: 4, dtype: object

In [6]:
data_x = data.drop(data.columns[4],axis=1)

In [7]:
data_x.columns = ['site','loc','faq','pagenum']

#数据预处理
已经得到x和targets数据
1、数值化
    one-hot
2、归一化

ROC曲线
特征之间协方差找线性相关系数

In [9]:
#[slashdot google digg kiwitobes (direct)] ->[1 2 3 4 5]
data_x.replace('slashdot',0, inplace=True)
data_x.replace('google',1, inplace=True)
data_x.replace('digg',2, inplace=True)
data_x.replace('kiwitobes',3, inplace=True)
data_x.replace('(direct)',4, inplace=True)

In [10]:
#[USA France UK New Zealand ] ->[1 2 3 4 5]
data_x.replace('USA',0, inplace=True)
data_x.replace('France',1, inplace=True)
data_x.replace('UK',2, inplace=True)
data_x.replace('New Zealand',3, inplace=True)

In [11]:
#[yes no ] ->[0 1]
data_x.replace('yes',1, inplace=True)
data_x.replace('no',0, inplace=True)

In [12]:
#0        None
# 1     Premium
# 2       Basic
data_targets.replace('None',0, inplace=True)
data_targets.replace('Premium',1, inplace=True)
data_targets.replace('Basic',2, inplace=True)

In [13]:
from sklearn import preprocessing
#导入数据处理模块，并生成one_hot()编码函数
enc = preprocessing.OneHotEncoder()
enc.fit(data_x.iloc[:,0:3])
x = enc.transform(data_x.iloc[:,0:3]).toarray()

In [14]:
x = pd.DataFrame(x)

In [15]:
#归一化
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
add = min_max_scaler.fit_transform(data_x['pagenum'].reshape(-1,1))
x[11] = add

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  after removing the cwd from sys.path.
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [16]:
from sklearn.cross_validation import train_test_split
#拆分训练数据与测试数据 
x_train, x_test, y_train, y_test = train_test_split(x,data_targets, test_size = 0.2)

In [17]:
####贝叶斯分类器####
#训练贝叶斯分类器
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB() 
clf.fit(x_train,y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [18]:
from sklearn.metrics import classification_report
#测试结果
answer = clf.predict(x_test)
print(x_test)
print(answer)
print(y_test)
print(np.mean( answer == y_test))
print(classification_report(y_test, answer, target_names = ['None', 'Premium','Basic']))

     0    1    2    3    4    5    6    7    8    9    10        11
11  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.583333
14  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.500000
13  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.750000
8   1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.583333
[0 1 0 2]
11    0
14    2
13    0
8     0
Name: 4, dtype: int64
0.5
             precision    recall  f1-score   support

       None       1.00      0.67      0.80         3
    Premium       0.00      0.00      0.00         0
      Basic       0.00      0.00      0.00         1

avg / total       0.75      0.50      0.60         4



D:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
